In [32]:
import pandas as pd
import numpy as np
import os
from sklearn.impute import SimpleImputer
from  sklearn.model_selection import  train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.multioutput import MultiOutputClassifier 
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [33]:
def load_dataset(file_path):
    return pd.read_sas(file_path, format="xport")

In [34]:
bp_data = load_dataset("BPXO_L.xpt")
bp_data

,SEQN,BPAOARM,BPAOCSZ,BPXOSY1,BPXODI1,BPXOSY2,BPXODI2,BPXOSY3,BPXODI3,BPXOPLS1,BPXOPLS2,BPXOPLS3
0,130378.0,b'R',4.0,135.0,98.0,131.0,96.0,132.0,94.0,82.0,79.0,82.0
1,130379.0,b'R',4.0,121.0,84.0,117.0,76.0,113.0,76.0,72.0,71.0,73.0
2,130380.0,b'R',4.0,111.0,79.0,112.0,80.0,104.0,76.0,84.0,83.0,77.0
3,130386.0,b'R',4.0,110.0,72.0,120.0,74.0,115.0,75.0,59.0,64.0,64.0
4,130387.0,b'R',4.0,143.0,76.0,136.0,74.0,145.0,78.0,80.0,80.0,77.0
...,...,...,...,...,...,...,...,...,...,...,...,...
7796,142306.0,b'R',2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7797,142307.0,b'R',4.0,127.0,75.0,132.0,73.0,131.0,72.0,71.0,70.0,67.0
7798,142308.0,b'R',3.0,106.0,65.0,106.0,69.0,112.0,74.0,58.0,61.0,69.0
7799,142309.0,b'R',3.0,127.0,81.0,125.0,82.0,128.0,81.0,80.0,79.0,83.0


In [ ]:
demo_data = load_dataset("DEMO_L.XPT")
demo_data

In [ ]:
fertin_data = load_dataset("FERTIN_L.xpt")
fertin_data

In [ ]:
insurance_data = load_dataset("HIQ_L.xpt")
insurance_data

In [ ]:
creative_data = load_dataset("HSCRP_L.xpt")
creative_data

In [ ]:
kidney_condition_data = load_dataset("KIQ_U_L.xpt")
kidney_condition_data

In [ ]:
BDmass_data = load_dataset("BMX_L.xpt")
BDmass_data

In [ ]:
vitD_data = load_dataset("VID_L.xpt")
vitD_data

In [ ]:
# Preg_data = load_dataset('UCPREG_L.xpt')
# Preg_data

In [ ]:
data = demo_data.merge(fertin_data, on='SEQN')
data = data.merge(creative_data, on = 'SEQN')
data = data.merge(kidney_condition_data, on = 'SEQN')
data = data.merge(insurance_data, on = 'SEQN')
data = data.merge(vitD_data, on = 'SEQN')
data = data.merge(BDmass_data, on = 'SEQN')
# data = data.merge(Preg_data, on = 'SEQN')
data

In [ ]:
data = data.drop_duplicates(subset='SEQN') 
data

In [ ]:
data.columns = data.columns.str.strip().str.lower()

In [ ]:
missing_percent = data.isnull().mean() 
cols_to_drop = missing_percent[missing_percent>0.5].index.tolist()
data = data.drop(columns = cols_to_drop)

missing_percent = data.insull().mean()
cols_to_drop = missing_percent[missing_percent>0.5].index.tolist()
data = data.drop(column = cols_to_drop)

In [ ]:
data

In [ ]:
numeric_cols = data.select_dtypes(include = ['int64', 'float64']).columns
categorical_cols = data.select_dtypes(include=['object']).columns
num_imputer = SimpleImputer(strategy = 'median')
data[numeric_cols] = numeric_imputer.fit_transform(data[numeric_cols])

In [ ]:
data

In [ ]:
filtered_data = data[data['riagendr'] != 1.0]
filtered_data

In [ ]:
filtered_data.columns

x= filtered_data.drop(columns=['hiq011','hiq032a','hiq210'])
y= data[['hiq011','hiq032a', 'hiq210']]
print("printing x values")
print(x)
print("printing y values")
print(y)

filtered_data.sum()

In [ ]:
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)  # scale the feature

In [ ]:
poly = PolynomialFeatures(degree=2, include_bias=False)
x_poly = poly.fit_transform(x_scaled)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.3, random_state=42)
print(x_train, y_train)

In [ ]:
model = LogisticRegression(class_weight='balanced', max_iter=10000)
model= MultiOutputClassifier(model)
model.fit(x_train, y_train)

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
# Evaluate the model
print("🔍 Accuracy per target label:")
for i, col in enumerate(y.columns):
    acc = accuracy_score(y_test[col], y_pred[:, i])
    print(f"{col}: {acc:.4f}")

for i, col in enumerate(y.columns):
    print(f"\n📊 Classification report for {col}:")
    print(classification_report(y_test[col], y_pred[:, i], zero_division=1))

In [ ]:
output_dir = "image"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [ ]:
plt.scatter(y_test, y_pred, alpha = 0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.title("LogisticRegression: Actual vs Predicted")
plt.show()

In [ ]:
png_path = os.path.join(output_dir, 'bias.png')
plt.savefig(png_path, bbox_inches='tight', dpi=300)
print(f"Pie chart exported as PNG to: {png_path}")

In [ ]:
for i, feature in enumerate(features_to_plot):
    if feature in filtered_data.columns:
        plt.subplot(1, len(features_to_plot), i + 1)
        sns.boxplot(y=filtered_data[feature])
        plt.title(f"{feature.upper()} Distribution")
        plt.ylabel(feature.upper())

plt.tight_layout()
plt.show()